In [ ]:
!pip -q install pillow transformers torch torchvision flask pyngrok==7.1.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 46.0 MB/s eta 0:00:00


In [ ]:
# <<< REQUIRED if you want a more stable tunnel >>>
#add your token
NGROK_AUTH_TOKEN = ""  # or leave "" to run without auth

from pyngrok import ngrok
if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN.strip():
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print("ngrok token set" if NGROK_AUTH_TOKEN else "no ngrok token provided (shorter sessions)")


ngrok token set


In [ ]:
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification

MODEL_NAME = "linkanjarad/mobilenet_v2_1.0_224-plant-disease-identification"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
model = AutoModelForImageClassification.from_pretrained(MODEL_NAME).to(device).eval()

# quick sanity check
len(model.config.id2label), list(model.config.id2label.items())[:3]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/transformers/models/mobilenet_v2/feature_extraction_mobilenet_v2.py:30: FutureWarning: The class MobileNetV2FeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use MobileNetV2ImageProcessor instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/9.34M [00:00<?, ?B/s]

(38, [(0, 'Apple Scab'), (1, 'Apple with Black Rot'), (2, 'Cedar Apple Rust')])

In [ ]:
from flask import Flask, request, jsonify
from PIL import Image
import io
import torch.nn.functional as F

app = Flask(__name__)

def predict_image(pil_img, top_k=1):
    # preprocess
    inputs = processor(images=pil_img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = F.softmax(logits, dim=-1)[0]

    # top-k
    k = max(1, min(top_k, logits.shape[-1]))
    values, indices = torch.topk(probs, k)
    labels = [model.config.id2label[int(i)] for i in indices]
    confidences = [float(v) for v in values]
    return labels, confidences

@app.route("/", methods=["GET"])
def health():
    return jsonify({"status": "ok", "model": MODEL_NAME})

@app.route("/predict", methods=["POST"])
def predict_route():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded. Send multipart/form-data with field name 'file'."}), 400
    file = request.files["file"]

    try:
        img = Image.open(io.BytesIO(file.read())).convert("RGB")
    except Exception as e:
        return jsonify({"error": f"Invalid image: {e}"}), 400

    # optional: top_k param (default 1)
    try:
        top_k = int(request.args.get("top_k", 1))
    except:
        top_k = 1

    labels, confidences = predict_image(img, top_k=top_k)
    response = {
        "disease": labels[0],
        "confidence": round(confidences[0], 4),
    }
    if top_k > 1:
        response["top_k"] = [
            {"label": l, "confidence": round(c, 4)}
            for l, c in zip(labels, confidences)
        ]
    return jsonify(response)


In [ ]:
from pyngrok import ngrok
import threading, time

# Make sure no previous tunnels are running
try:
    ngrok.kill()
except:
    pass

public_url = ngrok.connect(addr=5000, bind_tls=True)
print("🔗 Public URL:", public_url)

def run_flask():
    # host=0.0.0.0 to bind all interfaces inside the Colab VM
    app.run(host="0.0.0.0", port=5000, debug=False)

thread = threading.Thread(target=run_flask, daemon=True)
thread.start()

# Small wait so the server starts
time.sleep(2)
print("Flask server started.")


🔗 Public URL: NgrokTunnel: "https://dfd73c635e35.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


Flask server started.
